### 1. Crear el Script en Python:

Abrimos el editor de texto y creamos un nuevo archivo llamado ingest_script.py

Realizamos el codigo

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Configurar los parámetros de conexión
server_name = '186.129.9.177,1433'
database_name = 'COMERLOGISTICS'
username = 'Carlos'
password = 'Dapt01'
connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

# Directorio donde se encuentran los archivos CSV
data_directory = r'C:\Users\Administrator\Documents\CF\CIENCIA DE DATOS IA\Data Analytics\Trabajo Final\DataSet'

# Lista de archivos, tablas correspondientes y nombres de columna ID
files_and_tables = {
    'Tabla_Compras.csv': ('Tabla_Compras', 'CompraID'),
    'Tabla_Detallecompras.csv': ('Tabla_DetalleCompras', 'DetalleCompraID'),
    'Tabla_InventarioFinal.csv': ('Tabla_InventarioFinal', 'InventarioFinalID'),
    'Tabla_InventarioInicial.csv': ('Tabla_InventarioInicial', 'InventarioInicialID'),
    'Tabla_Producto.csv': ('Tabla_Producto', 'ProductoID'),
    'Tabla_VentasFinal.csv': ('Tabla_VentasFinal', 'VentaID')
}

# Función para cargar datos de forma incremental
def ingest_data():
    for file_name, (table_name, id_column) in files_and_tables.items():
        file_path = os.path.join(data_directory, file_name)
        if os.path.exists(file_path):
            # Leer el CSV en un DataFrame
            df = pd.read_csv(file_path)

            # Leer los datos existentes de la tabla en un DataFrame
            query = f"SELECT {id_column} FROM {table_name}"
            existing_ids = pd.read_sql(query, con=engine)[id_column]

            # Filtrar los datos nuevos
            new_data = df[~df[id_column].isin(existing_ids)]

            # Insertar solo los nuevos datos
            if not new_data.empty:
                new_data.to_sql(table_name, con=engine, if_exists='append', index=False)
                print(f"{len(new_data)} registros nuevos agregados a {table_name}")
            else:
                print(f"No hay nuevos registros para insertar en {table_name}")
        else:
            print(f"Archivo {file_name} no encontrado.")

# Ejecutar la ingesta de datos
ingest_data()


Guardamos el archivo como ingest_script.py en una ubicación accesible. C:\Users\Administrator\Documents\CF\CIENCIA DE DATOS IA\Data Analytics\Trabajo Final\Scripts.

### 2. Creamos un Archivo .bat para Ejecutar el Script

In [ ]:
@echo off
cd C:\Users\Administrator\Documents\CF\CIENCIA DE DATOS IA\Data Analytics\Trabajo Final\Scripts
python ingest_script.py


Guardamos el archivo como run_ingest_script.bat en la misma carpeta donde tenemos el script de Python

3. Configurar la Tarea Programada en Windows
Abrimos el Task Scheduler:

Presiona Win + R, escribimos taskschd.msc y presionamos Enter.
Esto abrirá el Task Scheduler.

Creamos una Nueva Tarea:

En el panel derecho, seleccionamos Create Basic Task....
Damos un nombre a la tarea, "Automatización de Ingesta de Datos Incremental", y hacemos clic en Next.

Configuramos el Trigger:

En la pantalla Trigger, seleccionamos Weekly y hacemos clic en Next.
Seleccionamos el día Sunday y establece la hora en 2:00 AM. (Para que se realice todas las semanas el domingo a las 2:00 AM.)
Nos aseguramos de que la tarea esté configurada para repetirse cada 1 semana.
Hacemos clic en Next.

Configurar la Acción:

En la pantalla Action, seleccionamos Start a program y hacemos clic en Next.
Hacemos clic en Browse... y seleccionamos el archivo .bat que creamos en el Paso 2.
Hacemos clic en Next.

Finalizar la Tarea:

Revisamos la configuración de la tarea y hacemos clic en Finish para crearla.

4. Validación y Monitorización
Probamos la Tarea Manualmente:

En el Task Scheduler, seleccionamos la tarea que acabamos de crear y hacemos clic en Run para probarla manualmente.
Verificamos que los datos se ingesten correctamente en la base de datos SQL Server.

Revisar los Resultados:

Verificamos los registros de la base de datos para asegurarnos de que los datos se cargaron correctamente y que no se duplicaron.

Habilitar Logging (Opcional):

Si se desea registrar las ejecuciones de la tarea, se puede agregar funciones de logging en el script de Python para guardar los resultados en un archivo de texto.